# 3F7 Lab: CamZIP

## Tree data structures

fvhgvg=1

Import all functions from the package trees where we put together a number of tools for handling trees in the 3F7 lab.

In [1]:
from trees import *

Now define a simple tree (play around with this command and construct more complicated trees....)

In [2]:
t = [-1,0,1,1,0]

The following command will print a string that can be copy-pasted into a tree visualising website like [phylo.io](https://phylo.io) (don't forget to add a new line at the end of the string after cutting and pasting)

In [3]:
print(tree2newick(t))
print('Cut and paste the string on the previous line and add a "new line" at the end of the string.')

(( ,),)
Cut and paste the string on the previous line and add a "new line" at the end of the string.


You can also add labels to the nodes in the `tree2newick` command.

In [4]:
print(tree2newick(t,['root', 'child 0', 'grandchild 0', 'grandchild 1', 'child 1']))

((grandchild 0,grandchild 1)child 0,child 1)root


If there are less labels than nodes, the labels will be interpreted "leaves first"

In [5]:
print(tree2newick(t,['symbol 0','symbol 1', 'symbol 2']))

((symbol 0,symbol 1)4,symbol 2)3


The following command converts a variable-length code described by a tree to a code table format.

In [6]:
print(tree2code(t))

{'0': [0, 0], '1': [0, 1], '2': [1]}


Verify that the inverse function can recover the tree. 

In [7]:
print(code2tree(tree2code(t)))

[-1, 0, 1, 1, 0]


But the following may happen as well. Can you explain why?

In [8]:
print(code2tree(tree2code([3,3,4,4,-1])))

[-1, 0, 1, 1, 0]


In [9]:
print(tree2newick([3,3,4,4,-1], ['grandchild 0', 'grandchild 1', 'child 0', 'child 1', 'root']))

(child 0,(grandchild 0,grandchild 1)child 1)root


Similarly but far more problematic is the following inversion. The resulting assignment of codeword to symbols is fundamentally different from the original and would result in wrong decoding.

In [10]:
print(tree2code(code2tree({'0':[1], '1':[0,1], '2':[0,0,1], '3':[0,0,0]})))

{'0': [0], '1': [1, 0], '2': [1, 1, 0], '3': [1, 1, 1]}


These problems are all solved when using the extended tree format.

In [11]:
xt = tree2xtree([3,3,4,4,-1], ['a', 'b', 'c'])
print(xt)

[[3, [], 'a'], [3, [], 'b'], [4, [], 'c'], [4, [0, 1], '3'], [-1, [2, 3], '4']]


In [12]:
print(xtree2code(code2xtree({'0':[1], '1':[0,1], '2':[0,0,1], '3':[0,0,0]})))

{'0': [1], '1': [0, 1], '2': [0, 0, 1], '3': [0, 0, 0]}


## Testing your Shannon-Fano Code

This next section can only be completed once you have a working Shannon-Fano function `shannon_fano()`

In [12]:
from vl_codes import shannon_fano
from random import random
p = [random() for k in range(16)]
p = dict([(chr(k+ord('a')),p[k]/sum(p)) for k in range(len(p))])
print(f'Probability distribution: {p}\n')
c = shannon_fano(p)
print(f'Codebook: {c}\n')
xt = code2xtree(c)
print(f'Cut and paste for phylo.io: {xtree2newick(xt)}')

Probability distribution: {'a': 0.01665843438004465, 'b': 0.11071718812174959, 'c': 0.11641796647837584, 'd': 0.09861027786812138, 'e': 0.1301300139236159, 'f': 0.032983231839595055, 'g': 0.03534298196923523, 'h': 0.03913293869192407, 'i': 0.018417755767463156, 'j': 0.026749056387669872, 'k': 0.07185154007140256, 'l': 0.02427626630662133, 'm': 0.01957427355660789, 'n': 0.10367649084533305, 'o': 0.06604716675323394, 'p': 0.0894144170390068}



TypeError: unsupported operand type(s) for +: 'int' and 'str'

We can upload data from a file, for example `hamlet.txt`, and display the first few lines...

In [13]:
f = open('hamlet.txt', 'r')
hamlet = f.read()
f.close()
print(hamlet[:294])

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


We now compute the startistics of the file:

In [15]:
from itertools import groupby
frequencies = dict([(key, len(list(group))) for key, group in groupby(sorted(hamlet))])
Nin = sum([frequencies[a] for a in frequencies])
p = dict([(a,frequencies[a]/Nin) for a in frequencies])
print(f'File length: {Nin}')

File length: 207039


We can view the alphabet of symbols used in the file:

In [14]:
print(list(p))

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']


We are now ready to construct the Shannon-Fano code for this file, and view its tree (cut and paste into [phylo.io](https://phylo.io), don't forget to add a carriage return at the end, click on "Branch Labels/Support" under "Settings", then right-click on the root of the tree and select "expand all". 

In [15]:
c = shannon_fano(p)
print(xtree2newick(code2xtree(c)))

TypeError: unsupported operand type(s) for +: 'int' and 'str'

Now we can actually encode the file `hamlet.txt` using the Shannon-Fano code we constructed.

In [18]:
from vl_codes import vl_encode
hamlet_sf = vl_encode(hamlet, c)
print(f'Length of binary sequence: {len(hamlet_sf)}')

NameError: name 'c' is not defined

We have commands to convert a bit sequence into a byte sequence (including a 3 bit prefix that helps us determine the length of the bit sequence):

In [19]:
from vl_codes import bytes2bits, bits2bytes
x = bits2bytes([0,1])
print([format(a, '08b') for a in x])
y = bytes2bits(x)
print(f'The original bits are: {y}')

['01101000']
The original bits are: [0, 1]


We now apply the bits to byte conversion to the compressed text of Hamlet to compute the length of the compressed file.

In [20]:
hamlet_zipped = bits2bytes(hamlet_sf)
Nout = len(hamlet_zipped)
print(f'Length of compressed string: {Nout}')

NameError: name 'hamlet_sf' is not defined

The compression ratio can be expressed in two ways, unitless or in bits/bytes:

In [21]:
print(f'Compression ratio (rateless): {Nout/Nin}')
print(f'Compression ratio (bits per byte): {8.0*Nout/Nin}')

NameError: name 'Nout' is not defined

The lower bound for compression is the Entropy, measured in bits, that can be computed using an in-line function in Python:

In [22]:
from math import log2
H = lambda pr: -sum([pr[a]*log2(pr[a]) for a in pr])
print(f'Entropy: {H(p)}')

Entropy: 4.449863631694343


We now proceed to decode the compressed Hamlet sequence

In [23]:
from vl_codes import vl_decode
xt = code2xtree(c)
hamlet_unzipped = vl_decode(hamlet_sf, xt)
print(f'Length of unzipped file: {len(hamlet_unzipped)}')

NameError: name 'c' is not defined

We can view the first few lines of the input (note the command `join` that turns the list of strings into one string)

In [24]:
print(''.join(hamlet_unzipped[:294]))

NameError: name 'hamlet_unzipped' is not defined

## Compressing and uncompressing files

This is where we put it all together, compressing directly from input to output file. Play around with these commands once you implemented Huffman coding and arithmetic coding. We begin by importing the compression and decompression functions.

In [25]:
from camzip import camzip
from camunzip import camunzip

The next commands define the method to be used and the filename. Modify those when you are trying other methods on various files. 

In [26]:
method = 'shannon_fano'
filename = 'hamlet.txt'

Now we do the actual compression and decompression...

In [27]:
camzip(method, filename)
camunzip(filename + '.cz' + method[0])

The next few lines perform various statistical measurements and verifies that the decompressed file is identical to the compressed file.

In [28]:
from filecmp import cmp
from os import stat
from json import load
Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
with open(filename + '.czp', 'r') as fp:
    freq = load(fp)
pf = dict([(a, freq[a]/Nin) for a in freq])
print(f'Entropy: {H(pf)} bits per symbol')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

Length of original file: 207039 bytes
Length of compressed file: 1 bytes
Compression rate: 3.8640062983302665e-05 bits/byte
Entropy: 4.449863631694343 bits per symbol
The files are different


## Huffman coding

This section will only work once you have a working function `huffman()`. We first repeat the tree construction and visualisation.

In [29]:
from vl_codes import huffman
xt = huffman(p)
print(xtree2newick(xt))

TypeError: '<' not supported between instances of 'tuple' and 'float'

Observe how the Huffman tree differs from the Shannon-Fano tree. What are its shortest and its longest codeword? You can use the `camzip` code above changing the method to `'huffman'` to test the compression rate etc. You may also want to do it by hand to test the error resilience:

In [30]:
c = xtree2code(xt)
hamlet_huf = vl_encode(hamlet, c)
hamlet_decoded = vl_decode(hamlet_huf, xt)
print(''.join(hamlet_decoded[:294]))

KeyError: ' '

We now introduce a random bit flip (bit 400 flipped) in the compressed sequence and observe the result.

In [31]:
hamlet_corrupted = hamlet_huf.copy()
hamlet_corrupted[400] ^= 1
hamlet_decoded = vl_decode(hamlet_corrupted, xt)
print(''.join(hamlet_decoded[:297]))

NameError: name 'hamlet_huf' is not defined

## Arithmetic coding

We first try "by hand" to operate the steps of arithmetic coding using floating point numbers. We first compute the cumulative probability distribution.

In [32]:
f = [0.0]
for a in p:
    f.append(f[-1]+p[a])
f.pop()
f = dict([(a,f[k]) for a,k in zip(p,range(len(p)))])

We now perform by hand the first `n=4` steps of arithmetic coding. Vary `n` to observe the loss of precision. 

In [33]:
lo, hi = 0.0, 1.0
n = 4
for k in range(n):
    a = hamlet[k]
    lohi_range = hi - lo
    hi = lo + lohi_range * (f[a] + p[a])
    lo = lo + lohi_range * f[a]
print(f'lo = {lo}, hi = {hi}, hi-lo = {hi-lo}')

lo = 0.03994572862162076, hi = 0.04551184792510039, hi-lo = 0.005566119303479632


The output sequence is roughly the binary expression of `lo` (not exactly) and we can compute and observe it. What length `ell` would we need when encoding all of Hamlet?

In [34]:
from math import floor, ceil
ell = ceil(-log2(hi-lo))+2 if hi-lo > 0.0 else 96
print(bin(floor(lo*2**ell)))

0b101000


We encode and decode Hamlet again using arithmetic coding and verify that the first few lines of the play look as expected.

In [36]:
import arithmetic as arith
arith_encoded = arith.encode(hamlet, p)
arith_decoded = arith.decode(arith_encoded, p, Nin)
print('\n'+''.join(arith_decoded[:294]))

4294967295
207039


We now repeat the steps above but introduce a one bit mistake (bit 399 flipped) and observe the effect on the decoded text. Repeat this experiment varying the location of the mistake or adding more than one mistake. What do you observe? Can you explain why?

In [36]:
arith_corrupted = arith_encoded.copy()
arith_corrupted[399] ^= 1
arith_decoded = arith.decode(arith_corrupted, p, Nin)
print('\n'+''.join(arith_decoded[:294]))

Arithmetic decoded 99%    
        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. I he   soeoW  Llon,o gn swt  s  .em urlo agdehnainf ts 
  aele 
 Nhhyr  a  Sh.ath rue oetantoboso nsiiho- eArpIntn      yd  qtr ayeth soon  he hh? Sesct aT  hLoT e n   Hysst 
o a gw sf  et deLtlucEir on  en  tawfiui 
